<h5 align="right">Yona Mellul</h5>
<h1 align="center">TME n°4</h1>
<h3 align="center">Programmation SAT - Générateur de championnat</h3>



## Exercice 2 - Modélisation

In [1]:
import time
import os

#### Fonctions utiles

In [2]:
def equipe(fichier_noms, num):
    '''
    renvoie le nom de l'equipe numéro num
    '''
    f = open(fichier_noms, "r")
    lines = f.readlines()
    nom = lines[num]
    return nom[:len(nom)-1]

def decoderListe(liste, nE):
    for elem in liste:
        j,x,y = decodage(elem, nE)
        print(elem, " : jour", j,", equipe", x, " vs. equipe", y)

def creerCNF(nE, nJ, chemin="./championnat.cnf"):

    f = open(chemin, "w")
    title = "c title\n"
    nl = "c\n"
    intro = "p cnf " + str(nbVar(nE, nJ)) +" "+ str(nbClauses(nE, nJ))+ "\n"
    c = encoder(nE,nJ)
    f.writelines([title, nl, intro, c])
    f.close()
    
    


### Question 1. Exprimer en fonction de ne et nj le nombre de variables propositionnelles utilisées

Le nombre de variables propositionelles utilisées est $nJ*nE*(nE - 1)$.

En effet, chaque jour, chaque équipe peut affronter toutes les autres équipes (sauf elle-même).

In [3]:
def nbVar(nE, nJ):
    return nJ*nE*(nE-1)
    

### Question 2. Ecrire une fonction de codage qui prend en argument ne, nj , j, x, y, et qui renvoie k

In [4]:
def getK(nE, nJ, j, x, y):
    return j*nE**2 + x*nE + y + 1

### Question 3. Ecrire une fonction de décodage qui retrouve j, x et y à partir de k, ne.

In [5]:
def decodage(k, nE):
    k = k-1
    y = k%nE
    k = k//nE
    x = k%nE
    j = k//nE
    
    return j, x, y

## Exercice 3 - Génération d’un planning de matchs

### Question 1. Contraintes de cardinalité.
Il s’agit ici de se doter de quelques fonctions pour faciliter l’encodage du problème avec des contraintes de cardinalité ”au plus 1” et ”au moins 1”. Pour la contrainte ”au plus 1”, on considère dans cet exercice un encodage par paires.

##### 1. Ecrire une fonction qui prend en argument une liste d’entiers et renvoie une clause, au format DIMACS, correspondant à la contrainte ”au moins une de ces variables est vraie” (Σivi ≥ 1).

In [6]:
def atLeastOne(liste):
    res = ""
    for elem in liste:
        res+= str(elem) +" "
    res += "0\n"
    return res 

In [7]:
print(atLeastOne([1, -2, 7, 5]))

1 -2 7 5 0



##### 2. Ecrire une fonction qui prend en argument une liste d’entiers et renvoie une clause, au format DIMACS, correspondant à la contrainte ”au plus une de ces variables est vraie” (Σivi >= 1).

In [8]:
def atMostOne(liste):
    n = len(liste)
    res = ""
    for i in range(n):
        for j in range(i, n):
            if i != j:
                res += str(-liste[i]) + " " + str(-liste[j]) + " 0\n"
                
    return res


In [9]:
print(atMostOne([1,-2, 3, 4]))

-1 2 0
-1 -3 0
-1 -4 0
2 -3 0
2 -4 0
-3 -4 0



### Question 2. Traduction du problème.
On s’attache maintenant à traduire le problème en une formule logique propositionnelle, en passant éventuellement par l’intermédiaire de contraintes de cardinalite.

##### 1. Traduire la contrainte C1 ”chaque équipe ne peut jouer plus d’un match par jour” en un ensemble de contraintes de cardinalité.

$\forall j \in \left\{ 0,1,...,nJ-1\right\}$, $\forall x \in \left\{ 0,1,...,nE-1\right\}, \sum_{y=0}^{nE} (m_{jxy}+m_{jyx})\leqslant{1}, {x}\neq{y}$


##### 2. Ecrire une fonction encoderC1 qui génère ces contraintes pour ne et nj donné.

In [10]:
def encoderC1(nE, nJ):
    res = ""
    for j in range(nJ):
        for x in range(nE):
            liste = []
            for y in range(nE):
                if x != y:
                    liste.append(getK(nE, nJ, j, x, y))
                    liste.append(getK(nE, nJ, j, y, x))

                    
            res += atMostOne(liste)
            
    return res

##### 3. Indiquer le nombre de contraintes et de clauses générées pour 3 équipes sur 4 jours et expliciter ces contraintes.

#### LE CAS GÉNÉRAL:

Pour chaque jour (nJ jours), chaque équipe (nE equipes) ne peut jouer ne peut jouer qu'au plus UN match (que ce soit à domicile ou à l'exterieur).
Donc il y'a $nJ*nE$ contraintes.

Pour chaque contrainte (equipe et jour fixé), on verifie que cette equipe a au maximum un match ce jour là (atMostOne). Donc, on vérifie que parmi les variables représentant cette equipe à ce jour (il y en a $2(nE-1)= 2nE-2$) , il n'y en ai qu'une maximum qui soit verifiée.

Cela donne donc $\frac{(2nE-2)*(2nE-3)}{2}$ clauses par contrainte.
On a $nJ*nE*\frac{(2nE-2)*(2nE-3)}{2}$ clauses au total pour les contraintes de types C1.

#### CAS AVEC 3 ÉQUIPES ET 4 JOURS:
On a donc 3*4 = 12 contraintes qui génèreront en tout 72 clauses.

In [11]:
def nbClausesC1(nE, nJ):
    return int(nJ * nE * ((2*nE-2) * (2*nE -3))/2)

In [12]:
nE = 3
nJ = 4
c = encoderC1(nE, nJ)
print("nb de clauses attendues = ", nbClausesC1(nE, nJ))
print("nb de clauses obtenues = ", c.count("0\n"))

nb de clauses attendues =  72
nb de clauses obtenues =  72


##### 4. Traduire la contrainte C2 ”Sur la durée du championnat, chaque équipe doit rencontrer l’ensemble des autres équipes une fois à domicile et une fois à l’extérieur, soit exactement 2 matchs par équipe adverse.” en un ensemble de contraintes de cardinalités.

$\forall x \in \left\{ 0,1,...,nE-1\right\}$, $\forall y \in \left\{ 0,1,...,nE-1\right\}, \sum_{j=0}^{nJ} m_{jxy}={1}$ et $\sum_{j=0}^{nJ}m_{jyx}={1}$


##### 5. Ecrire une fonction encoderC2 qui génère ces contraintes pour ne et nj donné.


In [13]:
def encoderC2(nE, nJ):
    res = ""
    l1 = []
    l2 = []
    for equipe in range(nE):        
        for y in range(equipe, nE):
            l1 = []
            l2 = []
            if equipe != y:
                for j in range(nJ):
                    l1.append(getK(nE, nJ, j, equipe, y))
                    l2.append(getK(nE, nJ, j, y, equipe))
            
                res += atLeastOne(l1) + atMostOne(l1)
                res += atLeastOne(l2) + atMostOne(l2)
    return res

##### 6. Indiquer le nombre de contraintes et de clauses générées pour 3 équipes sur 4 jours et expliciter ces contraintes.


#### LE CAS GÉNÉRAL:
Il y a deux contraintes par couples d'équipes differentes, afin que chaque équipe ait exactement 1 match a domicile et 1 match a l'exterieur avec toutes les autres equipes
Cela donne $\frac{nE(nE-1)}{2}*2 = nE*(nE-1)$ contraintes.

Chaque contrainte génère 1 clause pour le atLeastOne et $\frac{nJ(nJ-1)}{2}$ pour le atMostOne

Donc le nombre total de clauses est : $nE(nE-1)* (1+\frac{nJ(nJ-1)}{2}) clauses$


#### CAS AVEC 3 ÉQUIPES ET 4 JOURS:
On a donc 6 contraintes qui génereront 6*(1+6) = 42 clauses

In [14]:
def nbClausesC2(nE, nJ):
    return int(nE*(nE-1) * (1+((nJ*(nJ-1))/2)))

In [15]:
nE = 3
nJ = 4
c = encoderC2(nE, nJ)
print("nb de clauses attendues =", nbClausesC2(nE, nJ))
print("nb de clauses obtenues =", c.count("0\n"))

nb de clauses attendues = 42
nb de clauses obtenues = 42


##### 7. Ecrire un programme encoder qui encode toutes les contraintes C1 et C2 pour ne et nj donné.

In [16]:
def encoder(nE, nJ):
    return "c contraintes de types C1\n"+encoderC1(nE, nJ) +"c contraintes de types C2\n"+ encoderC2(nE, nJ)

In [17]:
def nbClauses(nE, nJ):
    return nbClausesC1(nE, nJ) + nbClausesC2(nE, nJ)

### Question 3. Utilisation du solveur.
##### Utiliser glucose sur la CNF générée à la question précédente et vérifier la première solution proposée pour 3 équipes sur 4 jours. Qu’est-il nécessaire d’ajouter aux deux contraintes C1 et C2 ?


Le solveur nous renvoie UNSAT. C'est normal. En effet, il y a 3 équipes. Donc chacune devrait jouer exactement 4 matchs (2 par équipes adverses). Or il y a seulement 4 jours de match, donc chaque équipe devrait jouer tous les jours (car un seul match par jour). Or, ce n'est pas possible. 

### Question 4. Décodage.
##### Ecrire une fonction ou un programme decoder qui prend pour argument un fichier contenant la sortie de l’appel à glucose (plus éventuellement nj et ne) et qui traduit le modèle rendu en une solution du problème de planning des matchs affichée lisiblement. On peut faire appel à un fichier extérieur donnant le nom des équipes : une par ligne dans leur ordre de numérotation.


In [18]:
def decoder(ficher, nE, nJ, fichier_noms= None):
    res = ""
    file = open(ficher, "r")
    ligne = file.readline()
    if ligne == "UNSAT\n":
        return None
    else:
        words = ligne.split()
        for w in words[:len(words)-1]:
            if w != 'c':
                i = int(w)
                if i>0 and w:
                    j,x,y = decodage(i, nE)
                    res+= "Le "+str(j+1)
                    if j == 0:
                        res += "er jour"
                    else:
                        res += "ème jour"
                    if fichier_noms != None:
                        res += ", l'équipe "+equipe(fichier_noms, x)+" affrontera l'équipe "+equipe(fichier_noms, y)+" à domicile. \n"
                    else:
                        res += ", l'équipe "+str(x+1)+" affrontera l'équipe "+str(y+1)+" à domicile. \n"

    file.close()
    return res

In [19]:
fichier_noms = "./noms_equipes.txt"

In [20]:
nE = 3
nJ = 6
creerCNF(nE, nJ)
os.system("glucose -model championnat.cnf championnat.txt")
print(decoder("./championnat.txt", nE, nJ, fichier_noms))
print(decoder("./championnat.txt", nE, nJ))

Le 1er jour, l'équipe DJ affrontera l'équipe IAMSI à domicile. 
Le 2ème jour, l'équipe IAMSI affrontera l'équipe DJ à domicile. 
Le 3ème jour, l'équipe DJ affrontera l'équipe ML à domicile. 
Le 4ème jour, l'équipe IAMSI affrontera l'équipe ML à domicile. 
Le 5ème jour, l'équipe ML affrontera l'équipe DJ à domicile. 
Le 6ème jour, l'équipe ML affrontera l'équipe IAMSI à domicile. 

Le 1er jour, l'équipe 3 affrontera l'équipe 1 à domicile. 
Le 2ème jour, l'équipe 1 affrontera l'équipe 3 à domicile. 
Le 3ème jour, l'équipe 3 affrontera l'équipe 2 à domicile. 
Le 4ème jour, l'équipe 1 affrontera l'équipe 2 à domicile. 
Le 5ème jour, l'équipe 2 affrontera l'équipe 3 à domicile. 
Le 6ème jour, l'équipe 2 affrontera l'équipe 1 à domicile. 



### Question 5. Assemblage final.
##### Ecrire un programme ou un script qui ´etant donné ne, nj et un fichier de noms d’équipes, affiche un planning des matchs en utilisant les programmes écrits dans les questions précédentes.

Il suffit de lancer l'exécutable *programme.py* qui vous demandera d'entrer le nombre d'équipes, le nombre de jours et, si vous en disposez d'un, le chemin vers le fichier des noms d'équipes. 

'>> python3 programme.py

## Exercice 4 - Optimisation du nombre de jours
En utilisant des appels SAT de façon itérative, indiquer comment trouver le nj minimal pour pouvoir planifier tous les matchs de championnat pour un nE donné.
Indiquer la valeur trouvée (ou une borne minimale) pour des ne croissants entre 3 et 10.
Si le solveur met plus de 10s à traiter le problème, on considère cela comme un Time Out (réponse inconnue).

In [21]:
def optimiser_nJ(nE_min = 1, nE_max = 4):
  
    nJ_best = [0 for i in range( nE_max - nE_min + 1 )] 
    
    for nE in range( nE_min, nE_max + 1 ):
        t_start = time.time()
        nJ = 1
        while time.time() - t_start < 10:
            source = "./championnat.cnf" #source tmp
            dest = "./championnat.txt" #destination tmp
            creerCNF(nE, nJ)
            commande = "glucose -model "+source+" "+dest
            os.system(commande)

            if decoder('./championnat.txt', nE, nJ) != None:
                nJ_best[ nE - nE_min ] = nJ
                break
            nJ += 1
            
    return nJ_best

In [22]:
optimiser_nJ()

[1, 2, 6, 6]